In [1]:
import pandas as pd
import re
from tqdm.notebook import tqdm

def get_second(time_str):
    segment = time_str.split(' ')
    second = 0    
    for i in segment:
        if i[-1] == 's':
            second += int(i[:-1])
        elif i[-1] == 'm':
            second += int(i[:-1]) * 60
        elif i[-1] == 'h':
            second += int(i[:-1]) * 60 * 60
        elif i[-1] == 'd':
            second += int(i[:-1]) * 60 * 60 * 24
        else:
            print(time_str)
    return second

df = pd.read_csv('../dataset/raw_dataset.csv',parse_dates=['created_at'])
df['is_bot'] = df['is_bot'].astype(bool)
print(df.dtypes)

id                          int64
change_id                  object
message_id                 object
author                      int64
real_author                 int64
created_at         datetime64[ns]
message                    object
revision_number             int64
is_bot                       bool
username                   object
dtype: object


In [2]:
## Get build status that change after recheck
tqdm.pandas()
output = dict()
CI_bots = dict()
df.sort_values(by=['created_at'], inplace=True)
build_count = dict()
CR = dict()

other_builds = dict()
other_CI_bots = dict()
other_CR = dict()
for key, group in tqdm(df.groupby(['change_id','revision_number'])):
    
    build_dict = dict()
    CI_bot = dict()
    build_fail, build_fail_then_recheck, recheck = False, False, False
    change_id , patchset = key
    
    for ind,col in group.iterrows():
    #start loop if build fail is found
        if not build_fail and col['is_bot'] and 'build' in col['message'].lower():
            build_fail = True

        if col['is_bot']:
            try:
                patch_fail = False
                build = re.findall(r'((?:merge|build) (?:succeeded|failed))',col['message'],re.IGNORECASE)

                if any('fail' in b.lower() for b in build): #or any('fail' in b.lower() for b in bot):
                    patch_fail = True
                elif any('abort' in b.lower() for b in build): #or any('abort' in b.lower() for b in bot):
                    patch_fail = True

                for message in col['message'].split('\n'):
                    build = re.findall(r'((?:post_failure|success|failure|aborted) in(?: \w+?)*s)',message,re.IGNORECASE)
                    if build:
                        build_name = message.split(' ')[1]
                        build_status = build[0].split(' in ')[0]
                        if build_name not in build_dict:
                            build_dict[build_name] = set()

                        # only consider failure to success story
                        if not recheck and (build_status.lower() == 'aborted' or build_status.lower() == 'failure'):
                            build_dict[build_name].add('FAILURE')
                    
                        if recheck and build_status == 'SUCCESS':
                            build_dict[build_name].add('SUCCESS')
                            
                        if build_name not in CI_bot:
                            CI_bot[build_name] = set()
                        CI_bot[build_name].add(col['username'])
            except:
                pass
        
        if 'recheck' in col['message'].lower() and col['is_bot'] == False:
            recheck = True
            
            if build_fail:
                build_fail_then_recheck = True

    if build_fail_then_recheck:
        for build in build_dict:            
            # change to == 1 and see consistant failure build
            if len(build_dict[build]) >= 2:
                if build not in build_count:
                    build_count[build] = 0
                build_count[build] += 1
                
                if build not in CI_bots:
                    CI_bots[build] = set()
            
                for ci in CI_bot[build]:
                    CI_bots[build].add(ci)
                
                if build not in CR:
                    CR[build] = set()
                
                CR[build].add(change_id)

  0%|          | 0/267239 [00:00<?, ?it/s]

In [3]:
output = list()
for key, value in sorted(build_count.items(), key=lambda item: item[1], reverse=True):
    if len(CI_bots[key]) > 1:
        continue
    
    output.append((key, value, sorted(CR[key])[0:10], sorted(CI_bots[key])))


In [4]:
# create DataFrame using data
df_output = pd.DataFrame(output, columns=['build name', 'Frequency', 'Code review list', 'bot name list'])
df_output.to_csv('falky_build.csv', index=False)